In [ ]:
!python -m spacy download en_core_web_sm
!pip install flair

In [116]:
import re
import spacy
import flair
import pandas as pd
from spacy import displacy
from tqdm.notebook import tqdm,trange
from collections import Counter

In [117]:
data=pd.read_json('../input/tweets-sentiment/tweets.json')
data=data.transpose()
data.reset_index(drop=True,inplace=True)

In [118]:
nlp = spacy.load('en_core_web_sm')
labels=[]
for index in trange(len(data)):
    txt=data['tweet_text'][index]
    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ not in labels:
            data[ent.label_]=pd.Series([[] for i in range(len(data))])
            data[ent.label_][index].append(ent.text)
            labels.append(ent.label_)
        else:
            if ent.text not in data[ent.label_][index]:
                data[ent.label_][index].append(ent.text)

  0%|          | 0/43347 [00:00<?, ?it/s]

In [119]:
def lower(text_array):
    ret_array=[]
    for item in text_array:
        item=re.sub("[^a-zA-Z0-9_]", '', item)
        item=re.sub(r'http\S+', '', item)
        ret_array.append(item.lower())
    return ret_array

low_cols=[col for col in list(data.columns) if col not in ['tweet_author','tweet_text']]
for col in low_cols:
    data[col]=data[col].apply(lower)

In [120]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [ ]:
def get_sentiment(text):
    preds=sentiment_analysis(text)
    label=preds[0]['label']
    score=preds[0]['score']
    
    if label == 'NEGATIVE':
        score = -1*score
    return pd.Series([label,score])

data[['sentiment','confidence']]=data['tweet_text'].apply(get_sentiment)

In [109]:
def counter(entity,top=None):
    """takes a colum of entity and counts number of unique entities returns entity counts or
        if top is given gets the top entity names
    """
    ret=[]
    for ent in entity:
        for item in ent:
            ret.append(item)
    count=Counter(ret)
    most=dict(count.most_common())
    del most['']
    
    if top is not None:
        top_items=[k for k, v in most.items() if v>=top]
        return top_items
    else:
        return most

['cll', 'nice']

In [115]:
def sentiment_entity(keys,col):
    """
    key=list of entity names
    col=domain of entity unique 
    possible domains=['ORG', 'PERCENT', 'CARDINAL', 'PERSON','LOC', 'GPE', 'MONEY', 'DATE',]
    """
    sents=[]
    for key in keys:
        index=[i for i in range(len(data)) if key in data[col][i]]
        result=data[['tweet_author','tweet_text','sentiment','confidence',col]].iloc[index]
        senti=result['confidence'].mean()
        sents.append(senti)
    return {keys[i]:sents[i] for i in range(len(keys))}

{'cll': 0.998074064652125, 'nice': 0.9977077394723892}

In [ ]:
top_orgs=counter(data['ORG'],top=10)
org_sentiments=sentiment_entity(top_orgs,'ORG')
org_sents